# Training a CatBoost regression model with data from DVC

With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. This notebook will guide you through an example that shows you how to build a Docker container for SageMaker and use it for training and inference.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies. 

### California Housing dataset
We use the California Housing dataset, present in [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html). 

The California Housing dataset was originally published in:

Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.


### DVC
DVC is built to make ML models shareable and reproducible. It is designed to handle large files, data sets, machine learning models, and metrics as well as code.

[DVC Official Site](https://dvc.org/)

# Part 1: Packaging and Uploading your Algorithm for use with Amazon SageMaker

### An overview of Docker

If you're familiar with Docker already, you can skip ahead to the next section.

For many data scientists, Docker containers are a new concept, but they are not difficult, as you'll see here. 

Docker provides a simple way to package arbitrary code into an _image_ that is totally self-contained. Once you have an image, you can use Docker to run a _container_ based on that image. Running a container is just like running a program on the machine except that the container creates a fully self-contained environment for the program to run. Containers are isolated from each other and from the host environment, so the way you set up your program is the way it runs, no matter where you run it.

Docker is more powerful than environment managers like conda or virtualenv because (a) it is completely language independent and (b) it comprises your whole operating environment, including startup commands, environment variable, etc.

In some ways, a Docker container is like a virtual machine, but it is much lighter weight. For example, a program running in a container can start in less than a second and many containers can run on the same physical machine or virtual machine instance.

Docker uses a simple file called a `Dockerfile` to specify how the image is assembled. We'll see an example of that below. You can build your Docker images based on Docker images built by yourself or others, which can simplify things quite a bit.

Docker has become very popular in the programming and devops communities for its flexibility and well-defined specification of the code to be run. It is the underpinning of many services built in the past few years, such as [Amazon ECS].

Amazon SageMaker uses Docker to allow users to train and deploy arbitrary algorithms.

In Amazon SageMaker, Docker containers are invoked in a certain way for training and a slightly different way for hosting. The following sections outline how to build containers for the SageMaker environment.

Some helpful links:

* [Docker home page](http://www.docker.com)
* [Getting started with Docker](https://docs.docker.com/get-started/)
* [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
* [`docker run` reference](https://docs.docker.com/engine/reference/run/)

[Amazon ECS]: https://aws.amazon.com/ecs/

### How Amazon SageMaker runs your Docker container

Because you can run the same image in training or hosting, Amazon SageMaker runs your container with the argument `train` or `serve`. How your container processes this argument depends on the container:

* In the example here, we don't define an `ENTRYPOINT` in the Dockerfile so Docker will run the command `train` at training time and `serve` at serving time. In this example, we define these as executable Python scripts, but they could be any program that we want to start in that environment.
* If you specify a program as an `ENTRYPOINT` in the Dockerfile, that program will be run at startup and its first argument will be `train` or `serve`. The program can then look at that argument and decide what to do.
* If you are building separate containers for training and hosting (or building only for one or the other), you can define a program as an `ENTRYPOINT` in the Dockerfile and ignore (or verify) the first argument passed in. 

#### Running your container during training

When Amazon SageMaker runs training, your `train` script is run just like a regular Python program. A number of files are laid out for your use, under the `/opt/ml` directory:

    /opt/ml
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <channel_name>
    |           `-- <input data>
    |-- model
    |   `-- <model files>
    `-- output
        `-- failure

##### The input

* `/opt/ml/input/config` contains information to control how your program runs. `hyperparameters.json` is a JSON-formatted dictionary of hyperparameter names to values. These values will always be strings, so you may need to convert them. `resourceConfig.json` is a JSON-formatted file that describes the network layout used for distributed training. Since scikit-learn doesn't support distributed training, we'll ignore it here.
* `/opt/ml/input/data/<channel_name>/` (for File mode) contains the input data for that channel. The channels are created based on the call to CreateTrainingJob but it's generally important that channels match what the algorithm expects. The files for each channel will be copied from S3 to this directory, preserving the tree structure indicated by the S3 key structure. 
* `/opt/ml/input/data/<channel_name>_<epoch_number>` (for Pipe mode) is the pipe for a given epoch. Epochs start at zero and go up by one each time you read them. There is no limit to the number of epochs that you can run, but you must close each pipe before reading the next epoch.

##### The output

* `/opt/ml/model/` is the directory where you write the model that your algorithm generates. Your model can be in any format that you want. It can be a single file or a whole directory tree. SageMaker will package any files in this directory into a compressed tar archive file. This file will be available at the S3 location returned in the `DescribeTrainingJob` result.
* `/opt/ml/output` is a directory where the algorithm can write a file `failure` that describes why the job failed. The contents of this file will be returned in the `FailureReason` field of the `DescribeTrainingJob` result. For jobs that succeed, there is no reason to write this file as it will be ignored.

#### Running your container during hosting

Hosting has a very different model than training because hosting is responding to inference requests that come in via HTTP. In this example, we use our recommended Python serving stack to provide robust and scalable serving of inference requests:

![Request serving stack](stack.png)

This stack is implemented in the sample code here and you can mostly just leave it alone. 

Amazon SageMaker uses two URLs in the container:

* `/ping` will receive `GET` requests from the infrastructure. Your program returns 200 if the container is up and accepting requests.
* `/invocations` is the endpoint that receives client inference `POST` requests. The format of the request and the response is up to the algorithm. If the client supplied `ContentType` and `Accept` headers, these will be passed in as well. 

The container will have the model files in the same place they were written during training:

    /opt/ml
    `-- model
        `-- <model files>



### The parts of the sample container

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageMager:

    .
    |-- Dockerfile
    `-- catboost_regressor
        |-- nginx.conf
        |-- predictor.py
        |-- serve
        |-- train
        `-- wsgi.py

Let's discuss each of these in turn:

* __`Dockerfile`__ describes how to build your Docker container image. More details below.
* __`catboost_regressor`__ is the directory which contains the files that will be installed in the container.
* __`local_test`__ is a directory that shows how to test your new container on any computer that can run Docker, including an Amazon SageMaker notebook instance. Using this method, you can quickly iterate using small datasets to eliminate any structural bugs before you use the container with Amazon SageMaker. We'll walk through local testing later in this notebook.

In this simple application, we only install five files in the container. You may only need that many or, if you have many supporting routines, you may wish to install more. These five show the standard structure of our Python containers, although you are free to choose a different toolset and therefore could have a different layout. If you're writing in a different programming language, you'll certainly have a different layout depending on the frameworks and tools you choose.

The files that we'll put in the container are:

* __`nginx.conf`__ is the configuration file for the nginx front-end. Generally, you should be able to take this file as-is.
* __`predictor.py`__ is the program that actually implements the Flask web server and the decision tree predictions for this app. You'll want to customize the actual prediction parts to your application. Since this algorithm is simple, we do all the processing here in this file, but you may choose to have separate files for implementing your custom logic.
* __`serve`__ is the program started when the container is started for hosting. It simply launches the gunicorn server which runs multiple instances of the Flask app defined in `predictor.py`. You should be able to take this file as-is.
* __`train`__ is the program that is invoked when the container is run for training. You will modify this program to implement your training algorithm.
* __`wsgi.py`__ is a small wrapper used to invoke the Flask app. You should be able to take this file as-is.

In summary, the two files you will probably want to change for your application are `train` and `predictor.py`.

In [43]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM public.ecr.aws/docker/library/python:3.7-slim

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         git \
         ca-certificates

RUN pip install numpy==1.16.2 scipy==1.2.1 catboost pandas flask gevent gunicorn
RUN pip install dvc==2.8.3 s3fs==2021.10.1 dvc-s3==2.8.3
RUN pip install awscli boto3
RUN pip install git-remote-codecommit

# Set some environment variables. PYTHONUNBUFFERED keeps Python from buffering our standard
# output stream, which means that logs can be delivered to the user quickly. PYTHONDONTWRITEBYTECODE
# keeps Python from writing the .pyc files which are unnecessary in this case. We also update
# PATH so that the train and serve programs are found when the container is invoked.

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBY

In the `container` directory are all the components you need to package the sample algorithm for Amazon SageMaker:

    .
    `-- container/
        |-- Dockerfile
        |-- README.md
        `--catboost_regressor/
            |-- nginx.conf
            |-- predictor.py
            |-- serve
            |-- train
            |-- wsgi.py


## Building and registering the container

TODO: explain what we are doing here and the `sm-docker` build command.

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-catboost-dvc

cd container

chmod +x catboost_regressor/train
chmod +x catboost_regressor/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-1 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

sm-docker build . --repository "${algorithm_name}:latest"

## Configure DVC for data versioning

Let us create a subdirectory where we prepare the data, i.e. `sagemaker-dvc-sample`.
Within this subdirectory, we initialize a new git repository and set the remote to a repository we create in AWS CodeCommit.
Finally, the `dvc` configurations and files for data tracking will be versioned in this repository.

One of the advantage of using AWS CodeCommit is its integration with IAM for authentication purposes, meaning we can use IAM roles to push / pull data without the need to fetch credentials or ssh keys. Setting the appropriate permissions on SageMaker execution role will also allow the SageMaker training job to interact securely with the AWS CodeCommit.

In [ ]:
%%sh

## Create the repository

repo_name="sagemaker-dvc-sample"

aws codecommit create-repository --repository-name ${repo_name} --repository-description "Sample repository to describe how to use dvc with sagemaker and codecommit"

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to eu-west-1 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

## repo_name is already in the .gitignore of the root repo

mkdir -p ${repo_name}
cd ${repo_name}

# initalize new repo in subfolder
git init
## Change the remote to the codecommit
git remote add origin https://git-codecommit."${region}".amazonaws.com/v1/repos/"${repo_name}"

# Configure git
git config --global user.email "you@example.com"
git config --global user.name "Your Name"

git config --global credential.helper '!aws codecommit credential-helper $@'
git config --global credential.UseHttpPath true

# Initialize dvc
dvc init

git commit -m 'Add dvc configuration'

# Set the DVC remote storage to S3
dvc remote add -d storage s3://sagemaker-"${region}"-"${account}"/DEMO-sagemaker-experiments-dvc
git commit .dvc/config -m "initialize DVC local remote"

# set the DVC cache to S3
dvc remote add s3cache s3://sagemaker-"${region}"-"${account}"/DEMO-sagemaker-experiments-dvc/cache
dvc config cache.s3 s3cache

# disable sending anonymized data to dvc for troubleshooting
dvc config core.analytics false

git add .dvc/config
git commit -m 'update dvc config'

git push --set-upstream origin master --force

### Import libraries and initial setup

In [23]:
import numpy as np
import pandas as pd
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = 'DEMO-sagemaker-experiments-dvc'

print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 583558296381
bucket: sagemaker-eu-west-1-583558296381
region: eu-west-1
role: arn:aws:iam::583558296381:role/RoleSagemakerStudioUsers


### Prepare raw data

Send the raw data to S3

In [28]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from pathlib import Path

databunch = fetch_california_housing()
dataset = np.concatenate((databunch["target"].reshape(-1, 1), databunch["data"]), axis=1)

print(f"Dataset shape = {dataset.shape}")
np.savetxt("dataset.csv", dataset, delimiter=",")

data_prefix_path = f"{prefix}/input/dataset.csv"
s3_data_path = f"s3://{bucket}/{data_prefix_path}"
print(f"Raw data location in S3: {s3_data_path}")

s3 = boto3.client("s3")
s3.upload_file("dataset.csv", bucket, data_prefix_path)

Dataset shape = (20640, 9)
s3://sagemaker-eu-west-1-583558296381/DEMO-sagemaker-experiments-dvc/input/dataset.csv


## Setup Experiments

Now, in order to track this test in Sagemaker, we need to create an experiment. We need to also define the trial within the experiment. For the sake of simplicity, we just consider one trial for the experiment, but we can have any number of trials within an experiment, for example if you want to test different algorithms.

In [ ]:
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

experiment_name = 'DEMO-sagemaker-experiments-dvc'

# create the experiment if it doesn't exist
try:
    my_experiment = Experiment.load(experiment_name=experiment_name)
    print("existing experiment loaded")
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        my_experiment = Experiment.create(
            experiment_name = experiment_name,
            description = "How to integrate DVC"
        )
        print("new experiment created")
    else:
        print(f"Unexpected {ex}=, {type(ex)}")
        print("Dont go forward!")
        raise

first_trial_name = "dvc-trial-v1"

try:
    my_first_trial = Trial.load(trial_name=first_trial_name)
    print("existing trial loaded")
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        my_first_trial = Trial.create(
            experiment_name=experiment_name,
            trial_name=first_trial_name,
        )
        print("new trial created")
    else:
        print(f"Unexpected {ex}=, {type(ex)}")
        print("Dont go forward!")
        raise

### Prepare processing job

### Version data with DVC

TODOs: install `dvc` related libraries within the script using `subprocess` calls (quick and dirty), or create a processing container where we have everything installed, possibly also configured (takes longer, but more elegant)

In [76]:
%%writefile preprocessing-experiment-1.py
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import os
import argparse
import sys
import subprocess

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# Read data locally
input_data_path = os.path.join("/opt/ml/processing/input", "dataset.csv")

prefix = '/opt/ml/'
input_path = prefix + 'processing/input'
dataset_path = prefix + input_path
train_channel_name = 'train'
validation_channel_name = 'validation'
base_dir = './sagemaker-dvc-sample/dataset'

train_path = os.path.join(dataset_path, train_channel_name)
validation_path = os.path.join(dataset_path, validation_channel_name)

#my_tracker = tracker.Tracker.load()

def clone_dvc_git_repo(dvc_repo_url):
    print(f"Configure git to pull authenticated from CodeCommit")
    print(f"Cloning repo: {dvc_repo_url}")
    subprocess.check_call(["git", "clone", dvc_repo_url])

def install_dependencies():
    print("Install git-remote-codecommit and sagemaker-experiments")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-Uq', 'git-remote-codecommit'])
    print("Install DVC and drivers for S3")
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-Uq', 'dvc==2.8.3', 's3fs==2021.10.1'])
    print("dependencies installed successfully")

def configure_git():
    print("Configure git")
    subprocess.check_call(['git', 'config', '--global', 'user.email', 'sagemaker-processing@example.com'])
    subprocess.check_call(['git', 'config', '--global', 'user.name', 'SageMaker Processing Job'])

def generate_train_validation_files(ratio):
    for path in ['train', 'validation', 'test']:
        output_dir = Path(f"{base_dir}/{path}/")
        output_dir.mkdir(parents=True, exist_ok=True)

    print("Read dataset")
    dataset = pd.read_csv(input_data_path)
    train, other = train_test_split(dataset, test_size=ratio)
    validation, test = train_test_split(other, test_size=ratio)
    
    print("create train, validation, test")
    pd.DataFrame(train).to_csv(f"{base_dir}/train/california_train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(f"{base_dir}/validation/california_validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(f"{base_dir}/test/california_test.csv", header=False, index=False)
    print("data created")

def sync_data_with_dvc(branch):
    os.chdir(base_dir)
    print(f"Create branch {branch}")
    try:
        subprocess.check_call(['git', 'checkout', '-b', branch])
        print(f"Create a new branch: {branch}")
    except:
        subprocess.check_call(['git', 'checkout', branch])
        print(f"Checkout existing branch: {branch}")
    print("Add files to DVC")
    subprocess.check_call(['dvc', 'add', 'train/california_train.csv'])
    subprocess.check_call(['dvc', 'add', 'validation/california_validation.csv'])
    subprocess.check_call(['dvc', 'add', 'test/california_test.csv'])
    subprocess.check_call(['git', 'add', '.'])
    subprocess.check_call(['git', 'commit', '-m', f"'add data for {branch}'"])
    print("Push data to DVC")
    subprocess.check_call(['dvc', 'push'])
    print("Push dvc metadata to git")
    subprocess.check_call(['git', 'push', '--set-upstream', 'origin', branch, '--force'])
    commit_hash = subprocess.check_output(['git', 'log', '--format=%H', '-n', '1']).decode("utf-8").replace('\n','')
    #my_tracker.log_parameters({"data_commit_hash": commit_hash})

if __name__=="__main__":
    parser = argparse.ArgumentParser()
    print(parser)
    parser.add_argument("--train-test-split-ratio", type=float, default=0.3)
    parser.add_argument("--dvc-repo-url", type=str, default="codecommit::eu-west-1://sagemaker-dvc-sample")
    parser.add_argument("--dvc-tag", type=str, default="v1.0.0")
    parser.add_argument("--dvc-branch", type=str, default="experiment-1")
    args, _ = parser.parse_known_args()
    
    install_dependencies()
    configure_git()
    clone_dvc_git_repo(args.dvc_repo_url)
    generate_train_validation_files(args.train_test_split_ratio)
    sync_data_with_dvc(args.dvc_branch)
    

Overwriting preprocessing-experiment-1.py


In [77]:
dvc_repo_url = "codecommit::{}://sagemaker-dvc-sample".format(region)
dvc_tag = "v1.0.0"

### Create processing container

In [87]:
!cat container/processing/Dockerfile

FROM public.ecr.aws/docker/library/python:3.7-slim

RUN apt-get -y update && apt-get install -y --no-install-recommends wget git

RUN pip3 install numpy pandas scikit-learn==1.0.2
RUN pip install awscli==1.22.67 boto3==1.21.12
RUN pip3 install sagemaker==2.77.1 sagemaker-experiments==0.1.35
RUN pip3 install git-remote-codecommit
RUN pip3 install dvc==2.8.3 s3fs==2021.10.1

RUN pip3 install git-remote-codecommit sagemaker-experiments

# Configure git

RUN git config --global user.email "sagemaker-processing@example.com"
RUN git config --global user.name "SageMaker ProcessingJob User"

ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]


In [84]:
%%sh

# The name of the image
image_name=sagemaker-processing-dvc

cd processor-container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-1 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

sm-docker build . --repository "${image_name}:latest"

...[Container] 2022/03/08 21:44:37 Waiting for agent ping

[Container] 2022/03/08 21:44:38 Waiting for DOWNLOAD_SOURCE
[Container] 2022/03/08 21:44:40 Phase is DOWNLOAD_SOURCE
[Container] 2022/03/08 21:44:40 CODEBUILD_SRC_DIR=/codebuild/output/src205974770/src
[Container] 2022/03/08 21:44:40 YAML location is /codebuild/output/src205974770/src/buildspec.yml
[Container] 2022/03/08 21:44:40 Setting HTTP client timeout to higher timeout for S3 source
[Container] 2022/03/08 21:44:40 Processing environment variables
[Container] 2022/03/08 21:44:40 No runtime version selected in buildspec.
[Container] 2022/03/08 21:44:40 Moving to directory /codebuild/output/src205974770/src
[Container] 2022/03/08 21:44:40 Configuring ssm agent with target id: codebuild:79af8537-ab9c-41eb-88ec-6a4806d0fca0
[Container] 2022/03/08 21:44:40 Successfully updated ssm agent configuration
[Container] 2022/03/08 21:44:40 Registering with agent
[Container] 2022/03/08 21:44:40 Phases found in YAML: 3
[Container] 2022/0

In [85]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type="ml.m5.large",
    instance_count=1
)

#ScriptProcessor to use a custom made image
#https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html

experiment_config={
    "ExperimentName": my_experiment.experiment_name,
    "TrialName": my_first_trial.trial_name,
    "TrialComponentDisplayName": "Training"
}

sklearn_processor.run(
    code='preprocessing-experiment-1.py',
    inputs=[ProcessingInput(source=s3_data_path, destination="/opt/ml/processing/input")],
    experiment_config=experiment_config
)


INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2022-03-08-21-47-34-529



Job Name:  sagemaker-scikit-learn-2022-03-08-21-47-34-529
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-583558296381/DEMO-sagemaker-experiments-dvc/input/dataset.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-west-1-583558296381/sagemaker-scikit-learn-2022-03-08-21-47-34-529/input/code/preprocessing-experiment-1.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []
...........................ArgumentParser(prog='preprocessing-experiment-1.py', usage=None, description=None, formatter_class=<class 'argparse.HelpFormatter'>, conflict_handler='error', add_help=True)
Install git-remote-codecommit 

UnexpectedStatusException: Error for Processing job sagemaker-scikit-learn-2022-03-08-21-47-34-529: Failed. Reason: AlgorithmError: See job logs for more information

In [ ]:
%%writefile preprocessing-experiment-2.py
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import os
import argparse
import sys
import subprocess

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# Read data locally
input_data_path = os.path.join("/opt/ml/processing/input", "dataset.csv")

prefix = '/opt/ml/'
input_path = prefix + 'processing/input'
dataset_path = prefix + input_path
train_channel_name = 'train'
validation_channel_name = 'validation'
base_dir = './sagemaker-dvc-sample/dataset'

train_path = os.path.join(dataset_path, train_channel_name)
validation_path = os.path.join(dataset_path, validation_channel_name)

#my_tracker = tracker.Tracker.load()

def clone_dvc_git_repo(dvc_repo_url):
    print(f"Cloning repo: {dvc_repo_url}")
    subprocess.check_call(["git", "clone", dvc_repo_url])

def generate_train_validation_files(ratio):
    for path in ['train', 'validation', 'test']:
        output_dir = Path(f"{base_dir}/{path}/")
        output_dir.mkdir(parents=True, exist_ok=True)

    print("Read dataset")
    dataset = pd.read_csv(input_data_path)
    train, other = train_test_split(dataset, test_size=ratio)
    validation, test = train_test_split(other, test_size=ratio)
    
    print("create train, validation, test")
    pd.DataFrame(train).to_csv(f"{base_dir}/train/california_train.csv", header=False, index=False)
    pd.DataFrame(validation).to_csv(f"{base_dir}/validation/california_validation.csv", header=False, index=False)
    pd.DataFrame(test).to_csv(f"{base_dir}/test/california_test.csv", header=False, index=False)
    print("data created")

def sync_data_with_dvc(branch):
    os.chdir(base_dir)
    print(f"Create branch {branch}")
    try:
        subprocess.check_call(['git', 'checkout', '-b', branch])
        print(f"Create a new branch: {branch}")
    except:
        subprocess.check_call(['git', 'checkout', branch])
        print(f"Checkout existing branch: {branch}")
    print("Add files to DVC")
    subprocess.check_call(['dvc', 'add', 'train/california_train.csv'])
    subprocess.check_call(['dvc', 'add', 'validation/california_validation.csv'])
    subprocess.check_call(['dvc', 'add', 'test/california_test.csv'])
    subprocess.check_call(['git', 'add', '.'])
    subprocess.check_call(['git', 'commit', '-m', f"'add data for {branch}'"])
    print("Push data to DVC")
    subprocess.check_call(['dvc', 'push'])
    print("Push dvc metadata to git")
    subprocess.check_call(['git', 'push', '--set-upstream', 'origin', branch, '--force'])
    commit_hash = subprocess.check_output(['git', 'log', '--format=%H', '-n', '1']).decode("utf-8").replace('\n','')
    #my_tracker.log_parameters({"data_commit_hash": commit_hash})

if __name__=="__main__":
    parser = argparse.ArgumentParser()
    print(parser)
    parser.add_argument("--train-test-split-ratio", type=float, default=0.3)
    parser.add_argument("--dvc-repo-url", type=str, default="codecommit::eu-west-1://sagemaker-dvc-sample")
    parser.add_argument("--dvc-tag", type=str, default="v1.0.0")
    parser.add_argument("--dvc-branch", type=str, default="experiment-1")
    args, _ = parser.parse_known_args()
    
    clone_dvc_git_repo(args.dvc_repo_url)
    generate_train_validation_files(args.train_test_split_ratio)
    sync_data_with_dvc(args.dvc_branch)

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-processing-dvc:latest".format(account, region)

script_processor = ScriptProcessor(command=['python3'],
                image_uri=image,
                role=role,
                instance_count=1,
                instance_type='ml.m5.xlarge')

#ScriptProcessor to use a custom made image
#https://docs.aws.amazon.com/sagemaker/latest/dg/processing-container-run-scripts.html

experiment_config={
    "ExperimentName": my_experiment.experiment_name,
    "TrialName": my_first_trial.trial_name,
    "TrialComponentDisplayName": "CustomProcessing"
}

script_processor.run(
    code='preprocessing-experiment-2.py',
    inputs=[ProcessingInput(source=s3_data_path, destination="/opt/ml/processing/input")],
    experiment_config=experiment_config
)


In [ ]:
%%sh

repo_name="sagemaker-dvc-sample"
cd "${repo_name}"

git checkout -b dataset_v1.0.0

dvc add dataset/test/california_test.csv
dvc add dataset/validation/california_validation.csv
dvc add dataset/train/california_train.csv

git add .

git commit -m 'add dev_dataset_1'

dvc push
git push --set-upstream origin dataset_v1.0.0

git tag v1.0.0
git push --tags

## Using your Algorithm in Amazon SageMaker

## Create an estimator and fit the model

To use DVC integration, pass a `dvc_repo_url` and `dvc_tag` as parameters when you create the Estimator object.

We will train on the `v1.0.0` tag first.

When doing `dvc pull`, this is the dataset structure:

```
dataset
    |-- train
    |   |-- california_train.csv
    |-- test
    |   |-- california_test.csv
    |-- validation
    |   |-- california_validation.csv
```

In [46]:
dvc_repo_url = "codecommit::{}://sagemaker-dvc-sample".format(region)
dvc_tag = "experiment-1"

In [47]:
dvc_repo_extended = "https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-dvc-sample".format(region)

with Tracker.create(display_name="DatasetLineage") as tracker:
    tracker.log_parameters(
        {
            "dataset_git_tag": dvc_tag,
            "dataset_git_repo": dvc_repo_extended
        }
    )

my_first_trial.add_trial_component(tracker.trial_component)

In [48]:
image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-catboost-dvc:latest".format(account, region)

metric_definitions = [{'Name': 'median-AE', 'Regex': "AE-at-50th-percentile: ([0-9.]+).*$"}]

estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    metric_definitions=metric_definitions,
    instance_type="ml.m5.large",
    sagemaker_session=sagemaker_session,
    hyperparameters={
            "dvc-repo-url": dvc_repo_url,
            "dvc-tag": dvc_tag
    },
)

experiment_config={
    "ExperimentName": my_experiment.experiment_name,
    "TrialName": my_first_trial.trial_name,
    "TrialComponentDisplayName": "Training"
}

In [49]:
estimator.fit(experiment_config=experiment_config)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-catboost-dvc-2022-03-08-14-30-06-951


2022-03-08 14:30:07 Starting - Starting the training job...
2022-03-08 14:30:22 Starting - Preparing the instances for trainingProfilerReport-1646749806: InProgress
......
2022-03-08 14:31:27 Downloading - Downloading input data......
2022-03-08 14:32:32 Training - Training image download completed. Training in progress.Reading hyperparameters data: /opt/ml/input/config/hyperparameters.json
hyperparameters_data: {'dvc-repo-url': 'codecommit::eu-west-1://sagemaker-dvc-sample', 'dvc-tag': 'experiment-1'}
Configure git to pull authenticated from CodeCommit
Cloning repo: codecommit::eu-west-1://sagemaker-dvc-sample, git tag: experiment-1
Cloning into '/opt/ml/input/data'...
remote: #015remote: Counting objects: 23#015remote: Counting objects: 23#015remote: Counting objects: 23, done.        
Running dvc pull command
A       validation/california_validation.csv
A       train/california_train.csv
A       test/california_test.csv
3 files added and 3 files fetched
Starting the training.
Found 

On the logs above you can see those lines, indicating about the files pulled by dvc:

```
Running dvc pull command
A       train/california_train.csv
A       test/california_test.csv
A       validation/california_validation.csv
3 files added and 3 files fetched
Starting the training.
Found train files: ['/opt/ml/input/data/dataset/train/california_train.csv']
Found validation files: ['/opt/ml/input/data/dataset/train/california_train.csv']
```

### Second version of the data


Data preparation

In [ ]:
## Data creation 

def split_dataframe(df, num=5):
    chunks = [df.iloc[i:i+num] for i in range(0,df.shape[0], int(df.shape[0] / num))]
    return chunks

for index, chunk in enumerate(split_dataframe(pd.DataFrame(train))):
    chunk.to_csv(f"{base_dir}/train/california_train_{index + 1}.csv", header=False, index=False)

for index, chunk in enumerate(split_dataframe(pd.DataFrame(validation), 3)):
    chunk.to_csv(f"{base_dir}/validation/california_validation_{index + 1}.csv", header=False, index=False)

In [ ]:
%%sh

repo_name="sagemaker-dvc-sample"
cd "${repo_name}"

git checkout -b dataset_v2.0.0

dvc add dataset/test/california_test*.csv
dvc add dataset/validation/california_validation*.csv
dvc add dataset/train/california_train*.csv

git add .

git commit -m 'add dev_dataset_2'

dvc push
git push --set-upstream origin dataset_v2.0.0

git tag v2.0.0
git push --tags

We will now train on the `v2.0.0` tag.

When doing `dvc pull`, this is the dataset structure:

```
dataset
    |-- train
    |   |-- california_train_1.csv
    |   |-- california_train_2.csv
    |   |-- california_train_3.csv
    |   |-- california_train_4.csv
    |   |-- california_train_5.csv
    |-- test
    |   |-- california_test.csv
    |-- validation
    |   |-- california_validation_1.csv
    |   |-- california_validation_2.csv
    |   |-- california_validation_3.csv
```

In [ ]:
second_trial_name = "dvc-trial-v2"

try:
    my_second_trial = Trial.load(trial_name=second_trial_name)
    print("existing trial loaded")
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        my_second_trial = Trial.create(
            experiment_name=experiment_name,
            trial_name=second_trial_name,
        )
        print("new trial created")
    else:
        print(f"Unexpected {ex}=, {type(ex)}")
        print("Dont go forward!")
        raise

In [ ]:
dvc_tag = "v2.0.0"

In [ ]:
with Tracker.create(display_name="DatasetLineage") as ptracker:
    ptracker.log_parameters(
        {
            "dataset_git_tag": dvc_tag,
            "dataset_git_repo": dvc_repo_extended
        }
    )

my_second_trial.add_trial_component(ptracker.trial_component)

In [ ]:
estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    metric_definitions=metric_definitions,
    instance_type="ml.m5.large",
    sagemaker_session=sagemaker_session,
    hyperparameters={
            "dvc-repo-url": dvc_repo_url,
            "dvc-tag": dvc_tag
        },
)

experiment_config={
    "ExperimentName": my_experiment.experiment_name,
    "TrialName": my_second_trial.trial_name,
    "TrialComponentDisplayName": "Training"
}

In [ ]:
estimator.fit(experiment_config=experiment_config)

On the logs above you can see those lines, indicating about the files pulled by dvc:

```
Running dvc pull command
A       validation/california_validation_2.csv
A       validation/california_validation_1.csv
A       validation/california_validation_3.csv
A       train/california_train_4.csv
A       train/california_train_5.csv
A       train/california_train_2.csv
A       train/california_train_3.csv
A       train/california_train_1.csv
A       test/california_test.csv
9 files added and 9 files fetched
Starting the training.
Found train files: ['/opt/ml/input/data/dataset/train/california_train_2.csv', '/opt/ml/input/data/dataset/train/california_train_5.csv', '/opt/ml/input/data/dataset/train/california_train_4.csv', '/opt/ml/input/data/dataset/train/california_train_1.csv', '/opt/ml/input/data/dataset/train/california_train_3.csv']
Found validation files: ['/opt/ml/input/data/dataset/validation/california_validation_2.csv', '/opt/ml/input/data/dataset/validation/california_validation_1.csv', '/opt/ml/input/data/dataset/validation/california_validation_3.csv']
```

## Hosting your model

In [ ]:
from sagemaker.predictor import csv_serializer

predictor = estimator.deploy(1, "ml.t2.medium", serializer=csv_serializer)

## Invoke endpoint with the Python SDK

In [ ]:
predicted = predictor.predict(test).decode('utf-8')
print(predicted)

### Delete the Endpoint

Make sure to delete the endpoint to avoid un-expected costs

In [ ]:
predictor.delete_endpoint()

### (Optional) Delete the Experiment, and all Trails, TrialComponents

In [ ]:
my_experiment.delete_all(action="--force")

### Delete the AWS CodeCommit repository

In [ ]:
!aws codecommit delete-repository --repository-name sagemaker-dvc-sample